In [1]:
print('Hello LangChain')

Hello LangChain


In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [ ]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

In [ ]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

In [5]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
content='LangServe는 개발자가 대규모 언어 모델(LLM)을 쉽게 배포하고 관리할 수 있도록 도와주는 플랫폼입니다. LangServe는 언어 모델의 개발, 테스트, 배포 및 운영을 간소화하여 개발자가 더 효율적으로 작업할 수 있도록 지원합니다.\n\nLangServe의 주요 기능은 다음과 같습니다:\n\n1. **언어 모델 배포**: LangServe를 사용하면 개발자가 언어 모델을 쉽게 배포할 수 있습니다. LangServe는 다양한 클라우드 플랫폼과 호환되며, 개발자는 자신의 언어 모델을 클라우드 환경에서 빠르게 배포할 수 있습니다.\n\n2. **모델 관리**: LangServe는 언어 모델의 버전을 관리하고, 모델의 성능을 모니터링하며, 필요에 따라 모델을 업데이트할 수 있는 기능을 제공합니다. 이를 통해 개발자는 모델의 성능을 지속적으로 개선할 수 있습니다.\n\n3. **API 제공**: LangServe는 언어 모델을 API로 제공하여 다른 애플리케이션에서 쉽게 사용할 수 있도록 합니다. 이는 개발자가 언어 모델을 다른 서비스에 통합할 때 매우 유용합니다.\n\n4. ** 보안 및 인증**: LangServe는 언어 모델의 배포 및 사용에 필요한 보안 및 인증 기능을 제공합니다. 이를 통해 개발자는 자신의 모델이 안전하게 사용되고 있는지 확인할 수 있습니다.\n\n5. **확장성**: LangServe는 수평적 확장성을 지원하여, 대규모 언어 모델도 빠르게 처리할 수 있습니다. 이는 대용량 데이터를 처리해야 하는 애플리케이션에 매우 중요합니다.\n\n6. **모니터링 및 로그**: LangServe는 모델의 성능과 사용량을 모니터링하고, 로그를 제공하여 문제 발생 시 빠르게 진단하고 해결할 수 있도록 돕습니다.\n\n7. **커뮤니티 지원**: LangServe는 개발자 커뮤니티를 지원하여, 사용자들끼리 경험을 공유하고 서로 도움을 줄 수 있는 플랫폼을

### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")\n    ')

In [7]:
# chain 연결 (LCEL) prompt + llm 연결
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### LCEL
* Prompt + LLM + OutputParser을 Chain으로 연결하기

In [8]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [9]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 컴퓨터가 데이터를 통해 스스로 학습하고 발전하는 과정입니다. 이는 마치 어린 아이가 사물을 인식하고 학습하는 방식과 유사합니다.

**기본 개념**

인공지능 모델은 크게 세 가지 요소로 구성됩니다.

1. **데이터**: 인공지능 모델이 학습하는 데 필요한 정보입니다. 데이터는 크게 **입력 데이터**와 **출력 데이터**로 나뉩니다.
2. **모델**: 데이터를 처리하고 학습하는 알고리즘입니다. 모델은 데이터를 분석하고 패턴을 찾아냅니다.
3. **손실 함수**: 모델의 성능을 평가하는 지표입니다. 손실 함수는 모델의 예측 결과와 실제 출력 데이터 간의 차이를 측정합니다.

**학습 과정**

인공지능 모델의 학습 과정은 다음과 같습니다.

1. **데이터 수집**: 다양한 소스에서 데이터를 수집합니다.
2. **데이터 전처리**: 수집한 데이터를 깨끗하고 사용 가능한 형태로 변환합니다.
3. **모델 초기화**: 모델을 초기화하고 학습을 시작합니다.
4. **예측**: 모델은 입력 데이터를 받아 예측 결과를 출력합니다.
5. **손실 계산**: 손실 함수를 통해 예측 결과와 실제 출력 데이터 간의 차이를 계산합니다.
6. **모델 업데이트**: 모델은 손실 함수를 최소화하기 위해 가중치를 업데이트합니다.
7. **반복**: 4~6단계를 반복하며 모델을 학습합니다.

**주요 학습 원리**

인공지능 모델의 학습 원리는 크게 두 가지로 나눌 수 있습니다.

1. **지도 학습**: 모델이 입력 데이터와 출력 데이터 쌍을 통해 학습하는 방식입니다. 모델은 입력 데이터를 받아 출력 데이터를 예측하고, 실제 출력 데이터와의 차이를 최소화합니다.
2. **비지도 학습**: 모델이 입력 데이터만으로 학습하는 방식입니다. 모델은 입력 데이터의 패턴이나 구조를 찾아냅니다.

**최적화 기법**

인공지능 모델의 학습에는 다양한 최적화 기법이 사용됩니다. 대표

In [10]:
# chain 호출
try:
    result = chain2.invoke({"input": ": LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe 같은 Product가 있어"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 인공지능(AI) 모델을 활용하여 자연어 처리(NLP) 및 대화형 AI 시스템을 구축하기 위한 플랫폼을 제공하는 회사입니다. LangChain의 주요 제품은 다음과 같습니다.

1. **LangSmith**: 랭스미스는 랭체인에서 제공하는 대화형 AI 모델 개발 및 관리 플랫폼입니다. 이 플랫폼을 통해 개발자는 대화형 AI 모델을 쉽게 구축, 테스트 및 배포할 수 있습니다. 랭스미스는 대화형 AI 모델의 성능을 평가하고 개선하는 데 도움이 되는 다양한 도구와 기능을 제공합니다.

2. **LangServe**: 랭서브는 랭체인에서 제공하는 API 기반의 대화형 AI 모델 배포 및 관리 서비스입니다. 이 서비스를 통해 개발자는 랭체인에서 지원하는 다양한 대화형 AI 모델을 쉽게 통합하고 배포할 수 있습니다. 랭서브는 대화형 AI 모델의 성능을 모니터링하고 관리하는 데 도움이 되는 다양한 기능과 도구를 제공합니다.

LangChain의 제품들은 대화형 AI 시스템을 구축하고 관리하는 데 필요한 다양한 도구와 기능을 제공하며, 개발자가 보다 쉽게 대화형 AI 모델을 구축하고 배포할 수 있도록 지원합니다.

LangChain의 제품들은 다음과 같은 특징을 가지고 있습니다.

* **쉬운 개발 및 배포**: 랭체인 제품들은 대화형 AI 모델을 쉽게 구축, 테스트 및 배포할 수 있는 도구와 기능을 제공합니다.
* **성능 평가 및 개선**: 랭체인 제품들은 대화형 AI 모델의 성능을 평가하고 개선하는 데 도움이 되는 다양한 도구와 기능을 제공합니다.
* **확장성 및 유연성**: 랭체인 제품들은 다양한 대화형 AI 모델을 지원하며, 개발자가 쉽게 통합하고 배포할 수 있습니다.

LangChain의 제품들은 대화형 AI 시스템을 구축하고 관리하는 데 필요한 다양한 요구사항을 충족시키며, 개발자가 보다 쉽게 대화형 AI 모델을 구축하고 배포할 수 있도록 지원합니다.


### Runnable의 stream() 함수 호출

In [11]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})
    
    # 스트리밍 출력
    #print(answer)
    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")


인공지능 모델의 학습 원리는 컴퓨터가 데이터를 통해 스스로 학습하고, 이를 바탕으로 예측이나 분류 작업을 수행하는 과정입니다. 이를 이해하기 위해, 먼저 인공지능 모델의 기본 개념과 학습 과정에 대해 설명하겠습니다.

### 1. 인공지능 모델이란?

인공지능 모델은 주어진 데이터로부터 학습을 통해 패턴이나 관계를 발견하고, 이를 기반으로 새로운 데이터에 대해 예측하거나 분류하는 알고리즘입니다. 이러한 모델에는 신경망, 결정 트리, 서포트 벡터 머신 등 여러 종류가 있습니다.

### 2. 학습 데이터

인공지능 모델의 학습은 데이터로부터 시작됩니다. 학습 데이터는 입력 데이터와 그에 대응하는 결과(레이블)로 구성됩니다. 예를 들어, 고양이와 개의 사진을 분류하는 모델을 학습시킬 때는, 사진(입력 데이터)과 사진에 대한 설명(레이블, 고양이 또는 개)이 필요합니다.

### 3. 모델의 학습 과정

모델의 학습 과정은 다음과 같습니다.

#### (1) 데이터 준비

- **데이터 수집**: 다양한 소스로부터 관련 데이터를 수집합니다.
- **데이터 전처리**: 데이터를 모델에 적합한 형태로 변환합니다. 여기에는 데이터 정리, 변환, 정규화 등이 포함됩니다.

#### (2) 모델 선택

- 문제의 성격에 따라 적합한 모델을 선택합니다. 예를 들어, 이미지 분류에는 합성곱 신경망(CNN), 자연어 처리에는 순환 신경망(RNN)이나 트랜스포머 등이 적합합니다.

#### (3) 모델 학습

- **모델 초기화**: 모델의 가중치를 무작위로 초기화합니다.
- **순전파(Forward Pass)**: 입력 데이터를 모델에 통과시켜 예측값을 얻습니다.
- **손실 함수 계산**: 예측값과 실제값(레이블) 사이의 차이를 계산합니다. 손실 함수는 모델의 성능을 평가하는 지표입니다.
- **역전파(Backward Pass)**: 손실 함수를 최소화하기 위해, 예측값과 실제값의 차이(오차)를 모델의 각 가중치에 대해 계산합니다. 이를 통해 가중치를 업데이트할 방향을 결정합니

### Multi Chain
* 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
* 두개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 한국 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 10문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [13]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

1.  **영화 제목**: 범죄도시2 (2022)

2.  **영화 줄거리 요약**:

    *   마석도는 서울 광수대와 협조하여 베트남 마약 조직에 대한 수사에 나선다.
    *   마석도는 광수대와 함께 베트남 마약 조직의 국내 잠복까지 수사에 나선다.
    *   마석도는 베트남 마약 조직의 국내 잠복을 조사하는 과정에서 여러 차례의 충돌을 겪는다.
    *   마석도는 광수대와 함께 베트남 마약 조직의 국내 잠복과 관련된 범죄자들을 체포하기 위해 노력한다.
    *   베트남 마약 조직의 국내 잠복을 조사하는 과정에서 마석도는 장첸과 그의 조직을 만나게 된다.
    *   마석도는 장첸과 그의 조직을 상대로 강력한 액션을 펼치며 수사를 진행한다.
    *   마석도는 장첸과 그의 조직을 상대로 치열한 대결을 벌이며 수사를 진행한다.
    *   마석도는 장첸과 그의 조직을 상대로 강력한 액션을 펼치며 많은 범죄자를 체포하는 데 성공한다.
    *   마석도는 장첸과 그의 조직을 상대로 치열한 대결을 벌이며 수사를 진행하고, 많은 범죄자를 체포하는 데 성공한다.
    *   마석도는 장첸과 그의 조직을 상대로 강력한 액션을 펼치며 많은 범죄자를 체포하는 데 성공하고, 수사는 마무리된다.


### PromptTemplate 여러개 연결하기

In [14]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

ChatGPT 모델의 학습 원리는 다음과 같습니다.

ChatGPT는 대규모 언어 데이터를 기반으로 학습된 인공지능 모델로, 주어진 문맥에 따라 적절한 응답을 생성할 수 있습니다. 이 모델은 강화 학습을 통해 인간과의 대화 데이터를 학습하고, 이를 통해 자연스러운 대화 흐름을 이해하고 응답을 생성합니다. 학습 과정에서 모델은 오류를 최소화하기 위해 지속적으로 업데이트되며, 이를 통해 성능이 향상됩니다.

ChatGPT 모델의 장점은 다음과 같습니다.

*   자연스러운 대화가 가능하며, 상황에 맞는 적절한 응답을 제공합니다.
*   대규모 언어 데이터를 기반으로 학습되어 다양한 주제와 상황에 대한 지식을 갖추고 있습니다.
*   사용자와의 상호작용을 통해 지속적으로 학습하고 성능을 향상시킬 수 있습니다.

ChatGPT 모델과 비슷한 AI 모델로는  LaMDA, LLaMA, PaLM 등이 있습니다.


In [15]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 요약해서 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 요약해서 한국어로 답변해 주세요.', 'llama-4 모델의 학습 원리를 4 문장으로 요약해서 한국어로 답변해 주세요.']


In [16]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) #AIMessage
    print(response.content)

GPT-4 모델은 대규모의 텍스트 데이터를 학습하여 언어 패턴과 구조를 익히는 방식으로 훈련됩니다. 이 모델은 주어진 문맥에서 다음에 올 가능성이 높은 단어를 예측하도록 스스로 학습하며, 이를 통해 자연스러운 텍스트를 생성하거나 대화에 참여할 수 있습니다.
Gemma는 컴퓨터가 자연어 처리를 더 잘하도록 설계된 인공지능(AI) 언어 모델입니다. 대규모의 텍스트 데이터를 학습하여 언어의 패턴과 구조를 이해하고, 이를 바탕으로 질문에 답변하거나 텍스트를 생성하는 등의 작업을 수행할 수 있습니다. Gemma는 사전 학습된 언어 모델을 미세 조정하여 특정 작업에 맞게 조정할 수 있으며, 이를 통해 성능을 더욱 향상시킬 수 있습니다.
llama-4 모델은 페이스북의 메타에서 개발한 대규모 언어 모델입니다. 이 모델은 수십억 개의 매개변수를 가지고 있으며, 방대한 양의 텍스트 데이터를 학습하여 자연어 처리 능력을 습득합니다. 학습 과정에서 모델은 주어진 문맥에서 다음에 올 단어를 예측하도록 훈련되며, 이를 통해 언어의 패턴과 구조를 학습합니다. 이를 통해 llama-4 모델은 다양한 자연어 처리 작업에 활용될 수 있습니다.


### ChatPromptTemplate 
* SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate 사용

In [17]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가입니다. 명확하고 자세하게 한국어로 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요?")

# LLM 호출
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 

딥러닝은 인공신경망을 기반으로 하는 머신러닝의 한 분야입니다. 인공신경망은 인간의 뇌를 모방한 구조로, 데이터를 처리하고 학습하는 방식입니다. 딥러닝은 이러한 인공신경망을 깊게 쌓아서 복잡한 데이터를 분석하고 학습하는 기술입니다.

딥러닝은 데이터에서 패턴을 발견하고, 이미지를 인식하거나, 자연어 처리를 수행하는 등 다양한 작업에 사용됩니다. 예를 들어, 이미지 인식, 음성 인식, 자연어 번역, 그리고 자율 주행 자동차 등의 분야에서 사용됩니다.

딥러닝의 핵심 아이디어는 데이터에서 특징을 자동으로 추출하고, 이를 통해 학습하는 것입니다. 이를 위해 딥러닝 모델은 여러 개의 레이어로 구성되어 있으며, 각 레이어는 데이터에서 특징을 추출하고 이를 다음 레이어로 전달합니다.

딥러닝의 대표적인 알고리즘으로는 합성곱 신경망 (Convolutional Neural Network, CNN), 순환 신경망 (Recurrent Neural Network, RNN), 그리고 심층 신경망 (Deep Neural Network, DNN) 등이 있습니다.

딥러닝은 많은 양의 데이터를 필요로 하며, 이를 통해 모델을 학습시키고 개선할 수 있습니다. 따라서 빅데이터와 컴퓨팅 파워가 필수적인 요소입니다.


### FewShotPromptTemplate
* 예시를 제공 프롬프트

In [19]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 태양과 가장 가까운 행성으로 매우 작고 온도가 극심하게 변합니다.
2. **금성**: 밝고 뜨거운 행성으로 강한 온실 효과로 표면 온도가 매우 높습니다.
3. **지구**: 생명체가 존재하는 유일한 행성으로 대기와 물이 있습니다.
4. **화성**: 붉은 행성으로 미래에 인간이 탐사할 가능성이 있습니다.
5. **목성**: 태양계에서 가장 큰 행성으로 가스 행성입니다.
6. **토성**: 아름다운 고리를 가진 가스 행성입니다.
7. **천왕성**: 빙하 행성으로 자전축이 기울어져 있습니다.
8. **해왕성**: 가장 먼 행성으로 강한 바람이 불고 있습니다.
